**Step 1: Extracting and Pre-processing Tweets**

In [ ]:
!pip install python-dotenv
!pip install tweet-preprocessor
!pip install flair
!pip install stanza

In [ ]:
# General dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import os
import time
from datetime import datetime, date, timedelta

# For Twitter API extraction
import tweepy
#!pip install python-dotenv
from dotenv import load_dotenv

# Tweet pre-processor
import preprocessor as p

# NLTK
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# TextBlob
from textblob import TextBlob

# FLairNLP
from flair.models import TextClassifier
from flair.data import Sentence

# Stanza
import stanza
stanza.download('en')

# Stanford CoreNLP
from pycorenlp import StanfordCoreNLP

In [ ]:
import tweepy

api_key = '' 
api_key_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

Step 2: Data Extraction with Twitter API and Text Pre-Processing

In [ ]:
# Generate list of dates (7 days window) based on today's date
list_of_dates = []
today = date.today()
for i in range(-7,1):
    target_date = (today + timedelta(days=i)).strftime("%Y-%m-%d")
    list_of_dates.append(target_date)

In [ ]:
list_of_dicts = []
search_term = 'covid19 covid vaccine covaxin covishield'
num_tweets = 16000

In [ ]:
def get_tweets(search_term = search_term, num_tweets = num_tweets):
    
    for end_date in list_of_dates:
        start_date = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=1)).strftime("%Y-%m-%d") # Create 1-day windows for extraction
        tweet_count = len(list_of_dicts)

        for tweet in tweepy.Cursor(api.search,
                                   q=f'{search_term} since:{start_date} until:{end_date}',
                                   lang = 'en',
                                   count = num_tweets,
                                   tweet_mode = 'extended').items(num_tweets):
            if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                if tweet.lang == "en":
                    tweet_dict = {}
                    tweet_dict['username'] = tweet.user.name
                    tweet_dict['location'] = tweet.user.location
                    tweet_dict['text'] = tweet.full_text
                    #tweet_dict['fav_count'] = tweet.favorite_count  
                    tweet_dict['hashtags'] = tweet.entities['hashtags']
                    tweet_dict['tweet_date'] = tweet.created_at
                    list_of_dicts.append(tweet_dict)
                    tweet_count +=1
                    print(f'Extracted tweet count = {tweet_count}')
                
        print(f'Completed extraction for {start_date} to {end_date}. Sleep for 10s')
        time.sleep(10)
        print('Ready to go again')

In [ ]:
# Run tweet extraction function
get_tweets()

In [ ]:
# Number of tweets pulled
len(list_of_dicts)

In [ ]:
# Transform list of dictionaries into a Pandas dataframe
tweets_df = pd.DataFrame(list_of_dicts)
tweets_df.sort_values(by='tweet_date').reset_index(drop = True)

In [ ]:
# Setup function to extract hashtags text from the raw hashtag dictionaries
def extract_hashtags(hashtag_list):
    
    s = "" # Create empty string
    if not hashtag_list: # If list is empty, return empty string
        return s
    else:
        for dictionary in hashtag_list:
            s+= str(dictionary['text'].lower() + ',') # Create string (lowercase) for each hashtag text
        s = s[:-1] # Drop last character ','
        return s

In [ ]:
# Extract hashtags
tweets_df['hashtags_extracted'] = tweets_df['hashtags'].apply(lambda x: extract_hashtags(x))
tweets_df.drop(columns = 'hashtags', inplace = True)

In [ ]:
tweets_df.head()

In [ ]:
# Keep only tweets that involve the vaccine
tweets_df_final = tweets_df[(tweets_df['text'].str.contains("vacc")) 
                            | (tweets_df['text'].str.contains("Vacc"))
                            | (tweets_df['hashtags_extracted'].str.contains("vacc"))
                            | (tweets_df['hashtags_extracted'].str.contains("Vacc"))]
len(tweets_df_final)

In [ ]:
# Create timestamp for datetime of extraction
extract_datetime = datetime.today().strftime('%Y%m%d_%H%M%S')

# Create csv filename
filename = 'Data/covid_vaccine_tweets_extracted_' + extract_datetime + '.csv'

# Drop duplicates (if any)
tweets_df_final.drop_duplicates(inplace = True)

# Export dataframe as csv file with above filename
tweets_df_final.to_csv(filename, index = False)

**Text Preprocessing**

In [ ]:
tweets_df = tweets_df_final.copy()

In [ ]:
# Clean tweet text with tweet-preprocessor
tweets_df['text_cleaned'] = tweets_df['text'].apply(lambda x: p.clean(x))

In [ ]:
# Remove duplicate tweets
tweets_df.drop_duplicates(subset='text_cleaned', keep="first", inplace = True)
len(tweets_df)

In [ ]:
# Remove unnecessary characters
# Note: Need to remove % as Stanford CoreNLP annotation encounters error if text contains some of these characters
punct =['%','/',':','\\','&amp;','&',';']

def remove_punctuations(text):
    for punctuation in punct:
        text = text.replace(punctuation, '')
    return text

tweets_df['text_cleaned'] = tweets_df['text_cleaned'].apply(lambda x: remove_punctuations(x))

In [ ]:
# Drop tweets which have empty text field
tweets_df['text_cleaned'].replace('', np.nan, inplace=True)
tweets_df['text_cleaned'].replace(' ', np.nan, inplace=True)
tweets_df.dropna(subset=['text_cleaned'], inplace=True)
len(tweets_df)

In [ ]:
tweets_df = tweets_df.reset_index(drop=True)
tweets_df.sample(5)

**Part 3 - Sentiment Analysis**

In [ ]:
# Define function to get value counts
def get_value_counts(col_name, analyzer_name):
    count = pd.DataFrame(tweets_df[col_name].value_counts())
    percentage = pd.DataFrame(tweets_df[col_name].value_counts(normalize=True).mul(100))
    value_counts_df = pd.concat([count, percentage], axis = 1)
    value_counts_df = value_counts_df.reset_index()
    value_counts_df.columns = ['sentiment', 'counts', 'percentage']
    value_counts_df.sort_values('sentiment', inplace = True)
    value_counts_df['percentage'] = value_counts_df['percentage'].apply(lambda x: round(x,2))
    value_counts_df = value_counts_df.reset_index(drop = True)
    value_counts_df['analyzer'] = analyzer_name
    return value_counts_df

**Part 3A - Sentiment Analysis with NLTK Vader**

Natural Learning Toolkit (NLTK) is a Python package that offers programs supporting natural language processing (NLP). In addition to its text corpus, it also comes with pre-trained models.  In particular, we will be using the Valence Aware Dictionary and Sentiment Reasoner (VADER) model, which is a lexicon and rule-based sentiment analysis tool specifically aimed at sentiment analysis of social media text. It uses a bag of words approach with simple heuristics (such as increasing sentiment intensity in presence of certain words like "very" or "really").

After installing NLTK with the command pip install nltk, we can run sentiment analysis using VADER with these lines of code:

In [ ]:
sia = SentimentIntensityAnalyzer()

# Obtaining NLTK scores
tweets_df['nltk_scores'] = tweets_df['text_cleaned'].apply(lambda x: sia.polarity_scores(x))

# Obtaining NLTK compound score
tweets_df['nltk_cmp_score'] = tweets_df['nltk_scores'].apply(lambda score_dict: score_dict['compound'])

In [ ]:
neutral_thresh = 0.05

In [ ]:
# Categorize scores into the sentiments of positive, neutral or negative
tweets_df['nltk_sentiment'] = tweets_df['nltk_cmp_score'].apply(lambda c: 'Positive' if c >= neutral_thresh else ('Negative' if c <= -(neutral_thresh) else 'Neutral'))

# Neutral score = 0
# tweets_df['nltk_sentiment'] = tweets_df['nltk_cmp_score'].apply(lambda c: 'Positive' if c > 0 else ('Negative' if c < 0 else 'Neutral'))

In [ ]:
tweets_df['nltk_cmp_score'].describe()

In [ ]:
nltk_sentiment_df = get_value_counts('nltk_sentiment','NLTK Vader')
nltk_sentiment_df

In [ ]:
sns.set_theme(style="dark")
ax = sns.barplot(x="sentiment", y="percentage", data=nltk_sentiment_df)
ax.set_title('NLTK Vader')

for index, row in nltk_sentiment_df.iterrows():
    ax.text(row.name,row.percentage, round(row.percentage,1), color='black', ha="center")

**Part 3B - Sentiment Analysis with TextBlob**

TextBlob is a popular Python library used to process textual data and perform a range of NLP tasks including sentiment analysis. Similar to NLTK Vader, the TextBlob sentiment classifier is also based on a bag of words approach. In fact, TextBlob is built upon the NLTK and pattern libraries.

This is the command to install TextBlob: pip install textblob

The NaiveBayesAnalyzer is trained on movies review dataset, so I will be using the default PatternAnalyzer instead

In [ ]:
# Obtain polarity scores generated by TextBlob
tweets_df['textblob_score'] = tweets_df['text_cleaned'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [ ]:
neutral_thresh = 0.05

In [ ]:
# Convert polarity score into sentiment categories
tweets_df['textblob_sentiment'] = tweets_df['textblob_score'].apply(lambda c: 'Positive' if c >= neutral_thresh else ('Negative' if c <= -(neutral_thresh) else 'Neutral'))

# Neutral score = 0
#tweets_df['textblob_sentiment'] = tweets_df['textblob_score'].apply(lambda c: 'Positive' if c > 0 else ('Negative' if c < 0 else 'Neutral'))

In [ ]:
tweets_df['textblob_score'].describe()

In [ ]:
textblob_sentiment_df = get_value_counts('textblob_sentiment','TextBlob')
textblob_sentiment_df

In [ ]:
sns.set_theme(style="dark")
ax = sns.barplot(x="sentiment", y="percentage", data=textblob_sentiment_df)
ax.set_title('TextBlob')

for index, row in textblob_sentiment_df.iterrows():
    ax.text(row.name,row.percentage, round(row.percentage,1), color='black', ha="center")

**Part 3C - Sentiment Analysis with Stanza**

Stanza is the default Python NLP library of the Stanford NLP Group, replacing the older Java-based CoreNLP. The modules are built on top of PyTorch, and its pre-built sentiment analyzer is trained on several datasets, including the Stanford Sentiment Treeback and Airline Twitter Sentiment.

Score mapping:
0: Negative
1: Neutral
2: Positive


In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment')

In [ ]:
def stanza_analyze(Text):
    document = nlp(Text)
    print('Processing')
    return np.mean([(i.sentiment - 1) for i in document.sentences]) # Minus 1 so as to bring score range of [0,2] to [-1,1]

In [ ]:
# Obtain sentiment categorical score generated by Stanza
tweets_df['stanza_score'] = tweets_df['text_cleaned'].apply(lambda x: stanza_analyze(x))

In [ ]:
tweets_df['stanza_score'].describe()

In [ ]:
neutral_thresh = 0.05

In [ ]:
# Convert average Stanza sentiment score into sentiment categories
tweets_df['stanza_sentiment'] = tweets_df['stanza_score'].apply(lambda c: 'Positive' if c >= neutral_thresh else ('Negative' if c <= -(neutral_thresh) else 'Neutral'))

In [ ]:
stanza_sentiment_df = get_value_counts('stanza_sentiment','Stanza')
stanza_sentiment_df

In [ ]:
sns.set_theme(style="dark")
ax = sns.barplot(x="sentiment", y="percentage", data=stanza_sentiment_df)
ax.set_title('Stanza')

for index, row in stanza_sentiment_df.iterrows():
    ax.text(row.name,row.percentage, round(row.percentage,1), color='black', ha="center")

Note: Other libraries (i.e. FlairNLP (Part 3D) and Stanford CoreNLP (Part 3E)) were also explored. FlairNLP was omitted from analysis as it does not give Neutral classification. Stanford CoreNLP was omitted from analysis as it has been replaced by Stanza as Stanford NLP Group's default NLP library.

**Part 3D - Sentiment Analysis with FlairNLP**

Flair pretrained sentiment analysis model is trained on IMDB movie reviews dataset

In [ ]:
flair_clf = TextClassifier.load('en-sentiment')

In [ ]:
# Create function to run flair sentiment analyzer
def flair_analyze(Text):
    sentence = Sentence(Text)
    flair_clf.predict(sentence)
    flair_senti = sentence.labels
    return flair_senti

In [ ]:
# Obtain sentiment scores generated by FlairNLP
tweets_df['flair_output'] = tweets_df['text_cleaned'].apply(lambda x: flair_analyze(x))

In [ ]:
tweets_df['flair_output'] = tweets_df['flair_output'].astype(str)

In [ ]:
def extract_flair_senti(output):
    sentiment = " ".join(re.findall("[a-zA-Z]+", output))
    sentiment = sentiment.capitalize()
    return sentiment

def extract_flair_score(output):
    score = ".".join(re.findall(r"\d+", output)) 
    return score

In [ ]:
# Get flair sentiment and polarity score
tweets_df['flair_sentiment'] = tweets_df['flair_output'].apply(lambda x:extract_flair_senti(x))
tweets_df['flair_score'] = tweets_df['flair_output'].apply(lambda x:extract_flair_score(x))

In [ ]:
flair_sentiment_df = get_value_counts('flair_sentiment','Flair')
flair_sentiment_df

In [ ]:
sns.set_theme(style="dark")
ax = sns.barplot(x="sentiment", y="percentage", data=flair_sentiment_df)
ax.set_title('Flair')

for index, row in flair_sentiment_df.iterrows():
    ax.text(row.name,row.percentage, round(row.percentage,1), color='black', ha="center")

On manual checking the approximate percentage of positive, negative and neutral tweets were found to be:

**Positive:** 39.473

**Negative:** 34.210

**Neutral:** 26.315